In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Lectura de data sets

In [ ]:
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

## Text preprocess

In [ ]:
!pip install symspellpy

In [ ]:
#Importamos librerias de pre procesamiento de texto
import nltk
import re
import string
from nltk.corpus import stopwords

import matplotlib as plt

from tqdm import tqdm
import spacy
from symspellpy.symspellpy import SymSpell, Verbosity
import pkg_resources
import json

In [ ]:
#utilizamos la libreria "re" para limpieza de texto
#uso funcion 
#re.sub(pattern, repl, string, count=0, flags=0)
#elimino corchetes, links, <>, puntuaciones, saltos de linea
#y remuevo las palabras que contienen numeros

#remuevo todas las puntuaciones salvo # y @
#no remuevo urls pq los pienso reemplazar mas adelante
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    punctuations_to_remove = string.punctuation.replace("#","")
    punctuations_to_remove = punctuations_to_remove.replace("@","")
    text = re.sub('[%s]' % re.escape(punctuations_to_remove), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

#Remuevo emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
def myStemmer(text):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    stemmer = nltk.stem.PorterStemmer()
    stemmed_list = []
    for token in tokens:
        stemmed_list.append(stemmer.stem(token))
    stemmed_text = ""
    separator = ' '
    stemmed_text = separator.join(stemmed_list)
    return stemmed_text

def myLemmatizer(text):
    tokenizer = nltk.tokenize.TreebankWordTokenizer()
    tokens = tokenizer.tokenize(text)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_list = []
    for token in tokens:
        lemmatized_list.append(lemmatizer.lemmatize(token))
    lemmatized_text = ""
    separator = ' '
    lemmatized_text = separator.join(lemmatized_list)
    return lemmatized_text
    

In [ ]:
#Separa la oracion en tokens y los lemmatiza, si son mentions @ o hasthtags # simplemente reemplaza estos simbolos
#por @MENTION y #HASHTAG segun corresponda
def _lemmatize_text(sentence, nlp):
    sent = ""
    doc = nlp(sentence)
    for token in doc:
        if '@' in token.text:
            sent+=" @MENTION"
        elif '#' in token.text:
            sent+= " #HASHTAG"
        else:
            sent+=" "+token.lemma_
    return sent

In [ ]:
def simplify(sentence):
    sent = _replace_urls(sentence)
    sent = _simplify_punctuation(sent)
    sent = _normalize_whitespace(sent)
    return sent

#Reemplaza todos los urls por "url"
def _replace_urls(text):
    url_regex = r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
    text = re.sub(url_regex, "<URL>", text)
    return text

#Elimina puntuaciones duplicadas
def _simplify_punctuation(text):
    corrected = str(text)
    corrected = re.sub(r'([!?,;])\1+', r'\1', corrected)
    corrected = re.sub(r'\.{2,}', r'...', corrected)
    return corrected

#remueve los espacios duplicados, de manera que haya solo 1 espacio entre palabras
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [ ]:
def normalize_contractions(text, contractions):
    """
    This function normalizes english contractions.
    """
    new_token_list = []
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        first_upper = False
        if word[0].isupper():
            first_upper = True
        if word.lower() in contractions:
            replacement = contractions[word.lower()]
            if first_upper:
                replacement = replacement[0].upper()+replacement[1:]
            replacement_tokens = replacement.split()
            if len(replacement_tokens)>1:
                new_token_list.append(replacement_tokens[0])
                new_token_list.append(replacement_tokens[1])
            else:
                new_token_list.append(replacement_tokens[0])
        else:
            new_token_list.append(word)
    sentence = " ".join(new_token_list).strip(" ")
    return sentence

In [ ]:
def init_spellchecker():
    max_edit_distance_dictionary= 3
    prefix_length = 4
    spellchecker = SymSpell(max_edit_distance_dictionary, prefix_length)
    dictionary_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_dictionary_en_82_765.txt")
    bigram_path = pkg_resources.resource_filename(
        "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
    spellchecker.load_dictionary(dictionary_path, term_index=0, count_index=1)
    spellchecker.load_bigram_dictionary(dictionary_path, term_index=0, count_index=2)
    return spellchecker

#Usando modulo pyspellchecker se hace una correcion ortografica simple
#Trabaja sobre texto tokenizado y solo la representacion de los tokens son alteradas.
def spell_correction(text, spellchecker):
    if len(text) < 1:
        return ""
    #Spell checker config
    max_edit_distance_lookup = 2
    suggestion_verbosity = Verbosity.TOP # TOP, CLOSEST, ALL
    #End of Spell checker config
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        if word is None:
            token_list[word_pos] = ""
            continue
        if not '\n' in word and word not in string.punctuation and not is_numeric(word) and not (word.lower() in spellchecker.words.keys()):
            suggestions = spellchecker.lookup(word.lower(), suggestion_verbosity, max_edit_distance_lookup)
            #Checks first uppercase to conserve the case.
            upperfirst = word[0].isupper()
            #Checks for correction suggestions.
            if len(suggestions) > 0:
                correction = suggestions[0].term
                replacement = correction
            #We call our _reduce_exaggerations function if no suggestion is found. Maybe there are repeated chars.
            else:
                replacement = _reduce_exaggerations(word)
            #Takes the case back to the word.
            if upperfirst:
                replacement = replacement[0].upper()+replacement[1:]
            word = replacement
            token_list[word_pos] = word
    return " ".join(token_list).strip()

#Elimina las letras que estan de mas en una palabra
#e.g yeeeeeeeeeeey -> yey
def _reduce_exaggerations(text):

    correction = str(text)
    #TODO work on complexity reduction.
    return re.sub(r'([\w])\1+', r'\1', correction)

def is_numeric(text):
    for char in text:
        if not (char in "0123456789" or char in ",%.$"):
            return False
    return True


In [ ]:
def text_preprocessing(text, nlp, contractions, spellchecker):
    text = simplify(text)
    text = normalize_contractions(text, contractions)
    text = spell_correction(text, spellchecker)
    #la limpieza la hago al final dado 
    #que remueve todas las puntuaciones y muchas son importantes
    #para la normalizacion del texto como (')
    text = clean_text(text)
    text = myStemmer(text)
    #text = myLemmatizer(text)
    #text = _lemmatize_text(text, nlp).strip()
    return text

## Agrego mas features

In [ ]:
# word_count
df_train['word_count'] = df_train['text'].apply(lambda x: len(str(x).split()))
df_test['word_count'] = df_test['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
df_train['unique_word_count'] = df_train['text'].apply(lambda x: len(set(str(x).split())))
df_test['unique_word_count'] = df_test['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
df_train['stop_word_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords.words('english')]))
df_test['stop_word_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stopwords.words('english')]))

# url_count
df_train['url_count'] = df_train['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
df_test['url_count'] = df_test['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
df_train['mean_word_length'] = df_train['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test['mean_word_length'] = df_test['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# char_count
df_train['char_count'] = df_train['text'].apply(lambda x: len(str(x)))
df_test['char_count'] = df_test['text'].apply(lambda x: len(str(x)))

# punctuation_count
df_train['punctuation_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test['punctuation_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
df_train['hashtag_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
df_test['hashtag_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
df_train['mention_count'] = df_train['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
df_test['mention_count'] = df_test['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

### Aplico funciones de preprocesamiento al data set

In [ ]:
#pre- procesamiento
df_train.head()

In [ ]:
df_train['text'][5]

In [ ]:
nlp = spacy.load('en')
spellchecker = init_spellchecker()
contraction_list = json.loads(open('../input/contractions/english_contractions.json', 'r').read())    
#Train set
df_train['text'] = df_train['text']\
    .apply(lambda x: text_preprocessing(x, nlp, contraction_list, spellchecker))
#Test set
df_test['text'] = df_test['text']\
    .apply(lambda x: text_preprocessing(x, nlp, contraction_list, spellchecker))

In [ ]:
df_train['text'][5]

In [ ]:
#post- procesamiento
df_train.head()

In [ ]:
df_test.head()

### (Opcional) Agrego los features pero despues de la limpieza

## Features extraction

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion

In [ ]:
#Feature extraction - BOW
count_vectorizer = CountVectorizer(ngram_range=(1,2))
train_bow = count_vectorizer.fit_transform(df_train['text'])
#print(count_vectorizer.get_feature_names())
test_bow = count_vectorizer.transform(df_test['text'])

In [ ]:
#TF IDF
tfidf_vectorizer = TfidfVectorizer()
train_tfidf = tfidf_vectorizer.fit_transform(df_train['text'])
#print(tfidf_vectorizer.get_feature_names())
test_tfidf = tfidf_vectorizer.transform(df_test['text'])

## XGBoost classifier

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import DMatrix

In [ ]:
#clf = XGBClassifier(random_state=42, seed=2, colsample_bytree=0.6, subsample=0.7)
clf = XGBClassifier(objective = 'binary:logistic', 
                    random_state=42, 
                    seed=2, 
                    colsample_bytree=0.5, 
                    subsample=0.7,
                    learning_rate=0.1,
                    )

## Crear Pipeline de features
Dado que tenemos uan combinacion de features numericos y de texto, uso funciones "helpers" que me devuelven la columna del texto o la columna del numero. Esto me permite alimentar el algoritmo con dos tipos de features distintos. Son parte del pipeline, se inicializan con el nombre de la columna de la data a ingresar. Tienen dos metodos, fit que devuelve la clase misma y transform retorna la columna con la data que corresponde al nombre de la columna

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None, *parg, **kwarg):
        return self

    def transform(self, X):
        # returns the input as a string
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # returns the input as a dataframe
        return X[[self.key]]

In [ ]:
text = Pipeline([
    ('selector', TextSelector(key='text')),
    ('vectorizer', count_vectorizer)])

word_count = Pipeline([
    ('selector',NumberSelector(key='word_count'))
])

unique_word_count = Pipeline([
    ('selector',NumberSelector(key='unique_word_count'))
])

stop_word_count = Pipeline([
    ('selector',NumberSelector(key='unique_word_count'))
])

url_count = Pipeline([
    ('selector',NumberSelector(key='url_count'))
])

mean_word_length = Pipeline([
    ('selector',NumberSelector(key='mean_word_length'))
])

char_count = Pipeline([
    ('selector',NumberSelector(key='char_count'))
])

punctuation_count = Pipeline([
    ('selector',NumberSelector(key='punctuation_count'))
])

hashtag_count = Pipeline([
    ('selector',NumberSelector(key='hashtag_count'))
])

mention_count = Pipeline([
    ('selector',NumberSelector(key='mention_count'))
])

In [ ]:
features = FeatureUnion([('text', text),
                        #('word_count', word_count),
                        ('unique_word_count', unique_word_count),
                        ('stop_word_count', stop_word_count),
                        #('url_count', url_count),
                        ('mean_word_length', mean_word_length),
                        #('char_count', char_count),
                        #('punctuation_count', punctuation_count),
                        #('hashtag_count', hashtag_count),
                        ('mention_count', mention_count)
                        ])

In [ ]:
pipe = Pipeline([('features', features),
                ('clf', clf)
                ])

In [ ]:
pipe.named_steps['clf'].get_params()

## Split train y validation set

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
combined_features = ['text', 
                    #'word_count', 
                    'unique_word_count',
                    'stop_word_count', 
                    #'url_count',
                    'mean_word_length',
                    #'char_count', 
                    #'punctuation_count', 
                    #'hashtag_count',
                    'mention_count'
                    ]
target = 'target'

X_train, X_test, y_train, y_test = train_test_split(df_train[combined_features], 
                                                    df_train[target], 
                                                    test_size=0.33, random_state=42, 
                                                    #stratify=df_train[target]
                                                     )
#print(X_train)
#print(y_train)
#print(X_test)
#print(y_test)

## Tuneo hiperparametros
Utilizando un grid search busco los mejores parametros para mi modelo, en este caso solo busco la cantidad de arboles n_estimators
agregar mas parametros incrementa considerablemente el tiempo de busqueda

In [ ]:
param_grid = {
     'clf__n_estimators': [50,100,300],
  #  'clf__colsample_bytree': [0.6,0.8,1]
#    'clf__subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1]
}

# grid search cross validation instantiation
grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid, 
                          cv = 5, n_jobs = 1, verbose = 0, return_train_score=True)

In [ ]:
#hyperparameter fitting
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.cv_results_['mean_train_score']

In [ ]:
grid_search.cv_results_['mean_test_score']

In [ ]:
grid_search.best_params_

## Prediccion train y validacion
#### score validacion (acertados para 2513 de validacion)

Todos tienen text
- tf-idf: 1939
- tf-idf + word_count: 1961
- tf_idf + todo: 1945
- bow: 1991
- bow + ngram(1,2) : 1992
- bow(ngram(1,2)) + word_count : 1991
- bow(ngram(1,2)) + word_count + unique_word_count : 1982
- bow(ngram(1,2)) + unique_word_count : 1996
- bow(ngram(1,2)) + unique_word_count + stop_words_count : 2004
- bow(ngram(1,2)) + unique_word_count + stop_words_count + url_count : 2000
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length : 2005
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + char_count : 1994
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + punctuation_count : 2005
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + hashtag_count: 1996
- bow(ngram(1,2)) + unique_word_count + stop_words_count + mean_word_length + mention_count: 2006

In [ ]:
clf_test = grid_search.best_estimator_

In [ ]:
pred_test = clf_test.predict(X_test)
#pred_test = clf.predict(X_test)

In [ ]:
#Hago un df con las prediciones y los y_test para compararlos
dic_compare_pred_test = {'pred_test':pred_test, 'y_test':y_test.values}

In [ ]:
df_compare_pred_test = pd.DataFrame(data=dic_compare_pred_test)

In [ ]:
df_compare_pred_test

In [ ]:
df_compare_pred_test['correct'] = (df_compare_pred_test['pred_test'] == df_compare_pred_test['y_test'])

In [ ]:
df_compare_pred_test.head()

In [ ]:
df_compare_pred_test['correct'].value_counts()

### Hiperparametros
Solo texto como feature
Los n_estimators no alteran el puntaje pq los defino con el gridSearchCV best_params
- random_state=42, seed=2, colsample_bytree=0.2, subsample=0.7: 1959
- random_state=42, seed=2, colsample_bytree=0.2, subsample=0.2: 1833
- random_state=42, seed=2, colsample_bytree=0.2, subsample=0.7, n_estimators=1000 : 1959
-  random_state=42, seed=2, colsample_bytree=0.2, subsample=0.7, n_estimators=1000, learning_rate=0.1 : 1973
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1 : **1982**
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.5 : 1893
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2 : 1982
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=10 : 1836
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=100 : 1699
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=1 : 1953
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.2, alpha=0.5 : 1961
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=0.8 : 1978
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=10 : 1892
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, n_estimators=1000, learning_rate=0.1, gamma=1 : 1977
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=15 : 1948
-  random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=5 : 1971
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=50 : 1929
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, min_child_weight=3 : 1920
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, min_child_weight=10 : 1843
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.02 : 1899
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.2 : 1971

#### bow(n_gram(1,2) 
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.01: 1852
- colsample_bytree=0.5, subsample=0.7, learning_rate=0.1 : 1992
- random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1, max_depth=1 : 1902
-  random_state=42, seed=2, colsample_bytree=0.5, subsample=0.7, learning_rate=0.1 : **2006**
                 
                   

## Predict

Despues de validarlo ahora hacemos la prediccion con el test set

In [ ]:
#obtengo los features del test set
data = df_test[combined_features]
data

In [ ]:
pred = clf_test.predict(data)

In [ ]:
pred

## Submission

In [ ]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

In [ ]:
sample_submission['target'] = pred
sample_submission.to_csv("submission.csv", index=False)
sample_submission.head(10)

In [ ]:
sample_submission.info()

## Verifico submission.csv

In [ ]:
df_sub = pd.read_csv("./submission.csv")
df_test_copy = df_test.copy(deep=True)
df_test_copy.head()

In [ ]:
#Copio el target de la prediccion al data set de test
df_test_copy['target'] = df_sub['target']
df_test_copy.head()